In [1]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
import pickle
import cv2
import random
import os



/home/mohamed/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
'''
        DO NOT RUN AGAIN
        reading the male,female data  
'''

male_path='/dataset/Male/Male'
female_path='/dataset/Female/Female'
#print(os.listdir(os.getcwd()+male_path))
male_path=os.getcwd()+male_path
female_path=os.getcwd()+female_path

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images
male_data=np.array(load_images_from_folder(male_path))

female_data=np.array( load_images_from_folder(female_path) )



In [3]:
print(male_data.shape)

print(female_data.shape)

(125,)
(133,)


In [4]:
## preprocess male and female  data 
for i,image in enumerate(male_data):
        image=female_data[i]
        image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #convert to gray
        image = cv2.GaussianBlur(image,(3,3),cv2.BORDER_DEFAULT)

        image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,11,2) # making the image binary
        image = cv2.medianBlur(image, 5)
        image = cv2.medianBlur(image, 5)
        image = cv2.erode(image, None, iterations = 1)
        female_data[i]=image
        #cv2.imwrite(f'lol{i}.png',image)  #saving the image to view them

        #doing the same again for the male data
        image=male_data[i]
        image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image = cv2.GaussianBlur(image,(3,3),cv2.BORDER_DEFAULT)
        image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,11,2)
        image = cv2.medianBlur(image, 5)
        image = cv2.medianBlur(image, 5)
        image = cv2.erode(image, None, iterations = 1)
        male_data[i]=image


In [8]:
for i,img in enumerate(female_data):
    cv2.imwrite("lol"+str(i)+".jpg",img)
    pass

In [ ]:
'''
to save data locally as numpy binary
'''

np.save('male_data.npy',male_data)
np.save('female_data.npy',female_data)

In [ ]:
'''
if you want to load the data again to work with them
'''

male_data=np.load('male_data.npy',allow_pickle=True)
female_data=np.load('female_data.npy',allow_pickle=True)

## ICDAR DATA


In [ ]:
df=pd.read_csv('train_answers.csv')
#print(df)

print(df['male'])
x=df['male'][3]
print("lol="+str(x))

binary_gender=np.array(df['male'])
print(binary_gender) #1 male   0 female


In [ ]:
new_images_path=os.getcwd()+'/images'

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images


In [ ]:
images_male=[]
images_female=[]
# k=10
# wshape=(100*k,142*k)
# image=cv2.resize(image,wshape)
new_images_path=os.getcwd()+'/images'

for filename in os.listdir(new_images_path):
    img=cv2.imread(os.path.join(new_images_path,filename))
    if img is not None:
        img_num=int(filename[:4])
        k=10
        wshape=(100*k,142*k)
        img=cv2.resize(img,wshape)
        #print(img_num)
        try:
            if binary_gender[img_num-1]:
                images_male.append(img)
                #print("male num:"+ str(img_num))
            else:
                images_female.append(img)
                #print("FEmale num:"+ str(img_num))
        except:
            pass    




In [ ]:
images_male=np.array(images_male)
images_female=np.array(images_female)


In [ ]:
def preprocess(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #convert to gray
    image = cv2.GaussianBlur(image,(3,3),cv2.BORDER_DEFAULT)

    image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2) # making the image binary
    image = cv2.medianBlur(image, 5)
    image = cv2.medianBlur(image, 5)
    image = cv2.erode(image, None, iterations = 1)
    return image
    


In [ ]:
male_processed=[]
female_processed=[]
for i,img in enumerate(images_male):
    male_processed.append(preprocess(img))
for i,img in enumerate(images_female):
    female_processed.append(preprocess(img))


In [ ]:
male_processed=np.array(male_processed)
female_processed=np.array(female_processed)

np.save('male_data_icdar.npy',male_processed)
np.save('female_data_icdar.npy',female_processed)
